In [10]:
from copy import deepcopy
import numpy as np
import pandas as pd
import calendar
from pybliometrics.scopus import ScopusSearch
from pybliometrics.scopus import AbstractRetrieval

from tqdm import tqdm

pd.options.display.max_columns = 30

filename = "./210803/BIPV_ML.txt"

In [82]:
### language
dic_language = {'eng': "English",
                'kor': "Korean"
               }


In [2]:
df_bipv = pd.read_pickle("./210803/df_bipv.pkl")
print(df_bipv.shape)
df_bipv.head()

(14385, 34)


,eid,doi,pii,pubmed_id,title,subtype,subtypeDescription,creator,afid,affilname,...,issueIdentifier,article_number,pageRange,description,authkeywords,citedby_count,openaccess,fund_acr,fund_no,fund_sponsor
0,2-s2.0-85110764627,10.1016/j.renene.2021.07.075,S0960148121010806,None,Markov chains estimation of the optimal period...,ar,Article,Sánchez-Barroso G.,60017838;126596986,Universidad de Extremadura;InnoDesarrollo S.L.,...,None,None,537-549,The European Dehesa has a very high potential ...,Cleaning frequency | Cleaning photovoltaic sys...,0,1,ESF,PD18047,European Social Fund
1,2-s2.0-85110299293,10.1016/j.energy.2021.121415,S0360544221016637,None,Stochastic modelling of variable renewables in...,ar,Article,Seljom P.,60004205;101126845,SINTEF Foundation for Scientific and Industria...,...,None,121415,None,Variable electricity generation from wind and ...,Energy-system modelling | Satellite-data | Sce...,0,0,None,268097,Norges Forskningsråd
2,2-s2.0-85107321575,10.1038/s41467-021-23592-0,None,34059672,Revealing composition and structure dependent ...,ar,Article,Lian W.,60089931;125236348,Hefei National Laboratory for Physical Science...,...,1,3260,None,Antimony trisulfide (Sb2S3) is a kind of emerg...,None,0,1,IUSS,22005293,National Outstanding Youth Science Fund Projec...
3,2-s2.0-85107176116,10.1038/s41598-021-91061-1,None,None,Combined nano and micro structuring for enhanc...,ar,Article,Perrakis G.,60072753;60029857;60001524,Texas A&amp;M University at Qatar;Foundation f...,...,1,11552,None,Outdoor devices comprising materials with mid-...,None,0,1,QNRF,NPRP9-383-1-083,Qatar National Research Fund
4,2-s2.0-85104778117,10.1038/s41598-021-87714-w,None,33883574,Decay radius of climate decision for solar pan...,ar,Article,Barton-Henry K.,60159447;60030162;60025182;60025038;60021763,Mercator Research Institute on Global Commons ...,...,1,8571,None,To design incentives towards achieving climate...,None,0,1,None,undefined,Volkswagen Foundation


In [3]:
df_rooftop = pd.read_pickle("./210803/df_rooftop.pkl")
print(df_rooftop.shape)
df_rooftop.head()

(921, 34)


,eid,doi,pii,pubmed_id,title,subtype,subtypeDescription,creator,afid,affilname,...,issueIdentifier,article_number,pageRange,description,authkeywords,citedby_count,openaccess,fund_acr,fund_no,fund_sponsor
0,2-s2.0-85110764627,10.1016/j.renene.2021.07.075,S0960148121010806,None,Markov chains estimation of the optimal period...,ar,Article,Sánchez-Barroso G.,60017838;126596986,Universidad de Extremadura;InnoDesarrollo S.L.,...,None,None,537-549,The European Dehesa has a very high potential ...,Cleaning frequency | Cleaning photovoltaic sys...,0,1,ESF,PD18047,European Social Fund
1,2-s2.0-85107176116,10.1038/s41598-021-91061-1,None,None,Combined nano and micro structuring for enhanc...,ar,Article,Perrakis G.,60072753;60029857;60001524,Texas A&amp;M University at Qatar;Foundation f...,...,1,11552,None,Outdoor devices comprising materials with mid-...,None,0,1,QNRF,NPRP9-383-1-083,Qatar National Research Fund
2,2-s2.0-85108691578,10.1016/j.seta.2021.101320,S2213138821003301,None,Techno-economic optimization for grid-friendly...,ar,Article,Christiaanse T.V.,60032882;60003122,Technische Universiteit Eindhoven;University o...,...,None,101320,None,High levels of non-dispatchable photo-voltaic ...,Bi-level optimization | British Columbia | Ene...,0,0,VU,undefined,Victoria University
3,2-s2.0-85107622660,10.1016/j.enconman.2021.114327,S0196890421005033,None,Coordinated energy-environmental-economic opti...,ar,Article,Hong Y.,60104720;60073652;60022195;60015138;113229129,University of Nottingham Ningbo China;Tongji U...,...,None,114327,None,Given that energy-efficiency policies focus on...,Life-cycle cost analysis | Low-rise | Office b...,0,0,None,undefined,None
4,2-s2.0-85105000579,10.1016/j.ijepes.2021.106932,S0142061521001721,None,Cool roof coating impact on roof-mounted photo...,ar,Article,Rahmani F.,60011442;126136083,Lamar University;Momentum Bay LLC,...,None,106932,None,Cool-roofing is effective in reducing cooling ...,Cool roof coating | DC-coupled solar photovolt...,0,0,NCI,undefined,National Cancer Institute


In [6]:
columns=["eid", "PT", "AU", "AF", "TI", 
         "SO", "SO_abb", "LA", "DT", "DE", 
         "ID", "AB", "C1", "RP", "EM", 
         "CR", "NR", "TC", "Z9", "SN", 
         "J9", "JI", "PD", "PY", "VL", 
         "AR", "DI", "SC"]

df_ab = pd.DataFrame(columns=columns)

In [16]:
starting_index = 0
data_index = list(range(starting_index, df_bipv.shape[0]))

for art in tqdm(data_index, desc="looping over df_bipv"):

    eid = df_bipv.loc[art, "eid"]
    ab = AbstractRetrieval(eid, view="FULL")

    # 1. PT: publication type
    docu_type_ = ab.srctype

    # 2. AU: index name
    index_name_ = [a.indexed_name for a in ab.authors] if ab.authors else [None]

    # 3. AF: author name
    author_name_ = [f"{a.surname}, {a.given_name}" for a in ab.authors] if ab.authors else [None]

    # 4. TI: document title
    docu_title_ = df_bipv.loc[art, "title"]

    # 5. SO: publication name
    src_title_ = ab.publicationName

    # 5-1. publication abbr.
    src_abb_ = ab.sourcetitle_abbreviation

    # 6. LA : Language
    try:
        language_ = dic_language[ab.language]
    except:
        language_ = "unknown"

    # 7. DT : Document Type
    docu_type_ = ab.subtype

    # 8. DE : Author Keywords
    auth_kw_ = ab.authkeywords

    if not auth_kw_:
        auth_kw_ = 'None'
    else:
        auth_kw_ = '; '.join(auth_kw_)

    # 9. ID : Keyword Plus
    kw_plus_ = 'None'

    # 10. AB : Abstract
    abstract_ = ab.abstract

    # 11. C1 : Author Address
#     if ab.authorgroup:
#         tmp = pd.DataFrame(ab.authorgroup)
#         grouped = tmp.groupby('organization').agg(lambda x: list(x))

#         aff_names = [str(aff) for aff in grouped["affiliation_id"].index.tolist()]
#         aff_ids = grouped["affiliation_id"].tolist()
#         citys = grouped["city"].tolist()
#         countrys = grouped["country"].tolist()
#         auids = grouped["auid"].tolist()
#         indexed_names = grouped["indexed_name"].tolist()

#         address = []
#         for aff_name, aff_id, city, country in zip(aff_names, aff_ids, citys, countrys):
#             if isinstance(aff_id, list):
#                 aff_id, city, country = aff_id[0], city[0], country[0]
#             address.append(f"{aff_id}, {aff_name}, {city}, {country}")

#         addresss_ = list(zip(auids, indexed_names, address))
    if ab.affiliation and ab.authors:
        df_aff = pd.DataFrame(ab.affiliation)
        df_aff["id"] = df_aff["id"].astype(str)
        df_auth = pd.DataFrame(ab.authors)
        df_authgroup = df_auth.groupby("affiliation").agg(list).reset_index()

        df_aff = pd.merge(df_aff, df_authgroup, left_on="id", right_on="affiliation").drop(["affiliation"], axis=1)
        df_aff["address"] = df_aff[['id', 'name', 'city', 'country']].apply(lambda x: ', '.join(x.astype(str)), axis=1)
        addresss_ = df_aff[["auid", "indexed_name", "address"]].values.tolist()
    else:
        addresss_ = []

    # df_aff = pd.DataFrame(ab.authorgroup)
    # grouped = df_aff.groupby("affiliation_id")
    # grouped[["organization", "affiliation_id", "city", "country"]]

    # 12. RP : Reprint Address
    rep_addr_ = "None"

    # 13. EM : E-mail Address
    em_addr_ = "None"

    # 14. CR : Cited References
    refs_ = []
    if ab.references != None:
        tmp = pd.DataFrame(ab.references)
        refcount = int(ab.refcount)

        for i in range(refcount):
            tmp_ = tmp.iloc[i]
            tmp_authors = tmp_['authors']
            if tmp_authors == None:
                tmp_authors = "[Anonymous]"
            tmp_year = tmp_['publicationyear']
            tmp_src = tmp_['sourcetitle']
            tmp_vol = tmp_['volume']
            tmp_page = tmp_['first']
            tmp_doi = tmp_['doi']

            ref = tmp_authors
            for item in [tmp_year, tmp_src, tmp_vol, tmp_page]:
                if item != None:
                    ref = ', '.join([ref, item])
            if tmp_doi != None:
                ref = ref + f", DOI {tmp_doi}"

            if i == 0:
                refs_.append(f"CR {ref}")
            else:
                refs_.append(f"   {ref}")

    # 15. NR : Cited Reference Count
    nr_ = ab.refcount

    # 16. TC : Web of Science Core Collection Times Cited Count
    tc_ = ab.citedby_count

    # 17. Z9 : Total Times Cited Count
    cc_ = tc_

    # 18. U1 : Usage Count (Last 180 Days)
    # 19. U2 : Usage Count (Since 2013)
    # 20. PU : Publisher = ELSEVIER SCI LTD
    # 21. PI : Publisher City = OXFORD
    # 22. PA : Publisher Address = THE BOULEVARD, LANGFORD LANE, KIDLINGTON, OXFORD OX5 1GB, OXON, ENGLAND
    # 23. SN : International Standard Serial Number (ISSN) = 0959-6526
    sn_ = "None" if ab.issn == None else ab.issn

    # 24. EI : Electronic International Standard Serial Number (eISSN) = 1879-1786
    # 25. J9 : 29-Character Source Abbreviation = J CLEAN PROD
    j9_ = ab.sourcetitle_abbreviation if ab.sourcetitle_abbreviation != None else "None"
    j9_ = j9_.upper()

    # 26. JI : ISO Source Abbreviation = J. Clean Prod.
    ji_ = ab.sourcetitle_abbreviation if ab.sourcetitle_abbreviation != None else "None"

    # 27. PD : Publication Date = JUL 1
    month = ab.coverDate.split('-')[1]
    date = ab.coverDate.split('-')[2]
    pd_ = f"{calendar.month_name[int(month)][:3].upper()} {int(date)}"

    # 28. PY : Publication Year = 2020
    py_ = ab.coverDate.split('-')[2]

    # 29. VL : Volumn = 260
    vl_ = ab.volume

    # 30. AR : Article Number = 121059
    ar_ = df_bipv.loc[art, "article_number"] if df_bipv.loc[art, "article_number"] != None else "None"

    # 31. DI : Digital Object Identifier = 10.1016/j.jclepro.2020.121059
    doi_ = ab.doi

    # 32. PG : Page Count = 14
    # 33. WC : Web of Science Categories = Green & Sustainable Science & Technology; Engineering, Environmental; Environmental Sciences
    # 34. SC : Research Areas = Science & Technology - Other Topics; Engineering; Environmental Sciences & Ecology
    if ab.subject_areas:        
        tmp = pd.DataFrame(ab.subject_areas)
        tmp_ = tmp["area"].tolist()
        sc_ = "; ".join(tmp_)
    else: 
        sc_ = [None]

    # 35. GA : Document Delivery Number = LL4XH
    # 36. UT : Accession Number = WOS:000531559900003
    # 37. DA : Date this report was generated. = 2020-06-14
    
    # summation
    data=[eid, docu_type_, index_name_, author_name_, docu_title_, 
                                src_title_, src_abb_, language_, docu_type_, auth_kw_, 
                                kw_plus_, abstract_, addresss_, rep_addr_, em_addr_, 
                                refs_, nr_, tc_, cc_, sn_, 
                                j9_, ji_, pd_, py_, vl_, 
                                ar_, doi_, sc_]
    
    df_ab = df_ab.append(dict(zip(columns, data)), ignore_index=True)

looping over df_bipv: 100%|██████████| 213/213 [02:19<00:00,  1.53it/s]


In [109]:
list_rooftop_eid = df_rooftop["eid"].tolist()
df_ab["is_rooftop"] = df_ab["eid"].apply(lambda x: True if x in list_rooftop_eid else False)

In [111]:
df_ab.to_pickle("./210803/df_ab.pkl")
df_ab.shape

(14385, 29)

In [99]:
# publication type
# "J" = Journal
# "B" = Book
# "S" = Series
# "P" = Patent
dict_pubtype = {'ar': 'J',    # 'Article',
                'ab': 'J',    # 'Abstract Report',
                'bk': 'B',    # 'Book',
                'ch': 'B',    # 'Book Chapter',
                'bz': 'J',    # 'Business Article',
                'cp': 'J',    # 'Conference Paper',
                'cr': 'J',    # 'Conference Review',
                'dp': 'J',    # 'Data Paper ',
                'ed': 'S',    # 'Editorial',
                'er': 'J',    # 'Erratum',
                'le': 'J',    # 'Letter',
                'no': 'S',    # 'Note',
                'pr': 'S',    # 'Press Release',
                'rp': 'J',    # 'Report',
                'tb': 'J',    # 'Retracted',
                're': 'J',    # 'Review',
                'sh': 'J',    # 'Short Survey'
                'ip': 'P'     # 'Patent' 
               }

# documentation type
dict_docutype = {'ar': 'Article',
                 'ab': 'Abstract Report',
                 'bk': 'Book',
                 'ch': 'Book Chapter',
                 'bz': 'Business Article',
                 'cp': 'Conference Paper',
                 'cr': 'Conference Review',
                 'dp': 'Data Paper ',
                 'ed': 'Editorial',
                 'er': 'Erratum',
                 'le': 'Letter',
                 'no': 'Note',
                 'pr': 'Press Release',
                 'rp': 'Report',
                 'tb': 'Retracted',
                 're': 'Review',
                 'sh': 'Short Survey',
                 'ip': 'Patent'
                }

# text cleaning
dict_clean_text = {"&amp;": "&", 
                   "&nbsp;": " ", 
                   "&lt;": "<",
                   "&gt;": ">"
                  }
def get_clean_text(text):
    for k, v in dict_clean_text.items():
        text = text.replace(k, v)
    return text

In [199]:
import os, calendar

with open(filename, "w") as datafile:
    datafile.write("FN Clarivate Analytics Web of Science\nVR 1.0")
    
    for art in df_ab.index:
        df_row = df_ab.loc[art]
        
        # 1. PT: publication type
        pubtype_ = dict_pubtype[df_row['PT']]
        datafile.write(f"\nPT {pubtype_}")
        
        # 2. AU: author names
        author_name_ = df_row['AU']
        author_name = "\nAU "
        if all(author_name_):
            author_name += "\n   ".join(author_name_)
            
        datafile.write(author_name)
        
        # 3. AF: affiliations
        affiliation_ = df_row['AF']
        affiliation = "\nAF"
        if all(affiliation_):
            affiliation += "\n   ".join(affiliation_)
        datafile.write(affiliation)
        
        # 4. TI: document title
        title = '\nTI '
        title_ = df_row['TI']
        if title_:
            title += get_clean_text(title_)
        datafile.write(title)
        
        # 5. SO: publication name
        so = '\nSO '
        so_ = df_row['SO']
        if so_:
            so += get_clean_text(so_)
        datafile.write(so)
        
        # 6. LA : Language
        datafile.write("\nLA " + df_row["LA"])
        
        # 7. DT : Document Type
        docutype_ = dict_docutype[df_row['DT']]
        datafile.write(f"\nDT {docutype_}")
        
        # 8. DE : Author Keywords
        de_ = df_row['DE']
        datafile.write(f"\nDE {de_}")
        
        # 9. ID : Keyword Plus
        id_ = df_row['ID']
        datafile.write(f"\nID {id_}")
        
        # 10. AB: Abstract
        ab_ = df_row['AB']
        datafile.write(f"\nAB {ab_}")
        
        # 11. C1 : Author Address
        c1_ = df_row['C1']
        c1 = [f"[{'; '.join(c[1])}] {', '.join(c[2].split(', ')[1:])}" for c in c1_]
        c1 = ".\n   ".join(c1) + "."
        datafile.write("\nC1 " + c1)
        
        # 12. RP : Reprint Address
        datafile.write("\nRP None")

        # 13. EM : E-mail Address
        datafile.write("\nEM None")
        
        # 14. CR : Cited References
        cr_ = df_row["CR"]
        if len(cr_) > 0:
            cr_[0] = "  " + cr_[0][2:]
            cr_ = [c for c in cr_ if len(c) > 5]
            cr_ = [c.lstrip(" ") for c in cr_ if (c[3] != ',') or ("DOI" in c)]
        datafile.write("\nCR "+"\n   ".join(cr_))
        
        # 15. NR : Cited Reference Count
        datafile.write(f"\nNR {df_row['NR']}")
        
        # 16. TC : Web of Science Core Collection Times Cited Count
        datafile.write(f"\nTC {df_row['TC']}")
        
        # 17. Z9 : Total Times Cited Count
        z9_ = df_row['Z9']
        datafile.write(f"\nZ9 {z9_}")
        
        # 18. U1 : Usage Count (Last 180 Days)
        # 19. U2 : Usage Count (Since 2013)
        # 20. PU : Publisher = ELSEVIER SCI LTD
        # 21. PI : Publisher City = OXFORD
        # 22. PA : Publisher Address = THE BOULEVARD, LANGFORD LANE, KIDLINGTON, OXFORD OX5 1GB, OXON, ENGLAND
        # 23. SN : International Standard Serial Number (ISSN) = 0959-6526
        sn_ = df_row["SN"]
        datafile.write(f"\nSN {sn_}")
        
        # 24. EI : Electronic International Standard Serial Number (eISSN) = 1879-1786
        # 25. J9 : 29-Character Source Abbreviation = J CLEAN PROD
        j9_ = df_row["J9"]
        datafile.write(f"\nJ9 {j9_}")
        
        # 26. JI : ISO Source Abbreviation = J. Clean Prod.
        ji_ = df_row["JI"]
        datafile.write(f"\nJI {ji_}")
        
        # 27. PD : Publication Date = JUL 1
        pd_ = df_row["PD"]
        datafile.write(f"\nPD {pd_}")
        
        # 28. PY : Publication Year = 2020
        py_ = df_row["PY"]
        datafile.write(f"\nPY {py_}")
        
        # 29. VL : Volumn = 260
        vl_ = df_row["VL"]
        datafile.write(f"\nVL {vl_}")
        
        # 30. AR : Article Number = 121059
        ar_ = df_row["AR"]
        datafile.write(f"\nAR {ar_}")
        
        # 31. DI : Digital Object Identifier = 10.1016/j.jclepro.2020.121059
        doi_ = df_row["DI"]
        datafile.write(f"\nDI {doi_}")
        
        # 32. PG : Page Count = 14
        # 33. WC : Web of Science Categories = Green & Sustainable Science & Technology; Engineering, Environmental; Environmental Sciences
        # 34. SC : Research Areas = Science & Technology - Other Topics; Engineering; Environmental Sciences & Ecology
        sc_ = df_row["SC"]
        datafile.write(f"\nSC {sc_}")
        
        # end 
        datafile.write(f"\nER\n")
        
    datafile.write("\nEF\n")

In [196]:
df_row["TI"]

In [73]:
df_ab.head(3)

,eid,PT,AU,AF,TI,SO,SO_abb,LA,DT,DE,ID,AB,C1,RP,EM,CR,NR,TC,Z9,SN,J9,JI,PD,PY,VL,AR,DI,SC
0,2-s2.0-85110764627,ar,"[Sanchez-Barroso G., Gonzalez-Dominguez J., Ga...","[Sánchez-Barroso, Gonzalo, González-Domínguez,...",Markov chains estimation of the optimal period...,Renewable Energy,Renew. Energy,English,ar,Cleaning frequency; Cleaning photovoltaic syst...,None,© 2021 The AuthorsThe European Dehesa has a ve...,"[[[57216405703, 57216406595, 57197703890], [Sa...",None,None,"[CR , 2012, , 2020, Spain's Electricity Gri...",44,0,0,18790682 09601481,RENEW. ENERGY,Renew. Energy,DEC 1,01,179,None,10.1016/j.renene.2021.07.075,"Renewable Energy, Sustainability and the Envir..."
1,2-s2.0-85110299293,ar,"[Seljom P., Kvalbein L., Hellemo L., Kaut M., ...","[Seljom, Pernille, Kvalbein, Lisa, Hellemo, La...",Stochastic modelling of variable renewables in...,Energy,Energy,English,ar,Energy-system modelling; Satellite-data; Scena...,None,© 2021 Elsevier LtdVariable electricity genera...,"[[[26767755100, 20436233400, 57196488296], [He...",None,None,"[CR Collins, S.; Deane, J.P.; Poncelet, K.; Pa...",62,0,0,03605442,ENERGY,Energy,DEC 1,01,236,121415,10.1016/j.energy.2021.121415,Civil and Structural Engineering; Building and...
2,2-s2.0-85107321575,ar,"[Lian W., Jiang C., Yin Y., Tang R., Li G., Zh...","[Lian, Weitao, Jiang, Chenhui, Yin, Yiwei, Tan...",Revealing composition and structure dependent ...,Nature Communications,Nat. Commun.,English,ar,None,None,"© 2021, The Author(s).Antimony trisulfide (Sb2...","[[[57203962991, 57196472762, 55809691800, 5718...",None,None,"[CR Park, J.S.; Kim, S.; Xie, Z.; Walsh, A., 2...",25,0,0,20411723,NAT. COMMUN.,Nat. Commun.,DEC 1,01,12,3260,10.1038/s41467-021-23592-0,"Chemistry (all); Biochemistry, Genetics and Mo..."


In [181]:
df_row["CR"]

[]

In [161]:
cr_ = [c for c in df_ab.loc[0, "CR"] if len(c) > 5]
cr = [c.lstrip(" ") for c in cr_ if (c[3] != ',') or ("DOI" in c)]
cr

['Khonkar, H.; Alyahya, A.; Aljuwaied, M.; Halawani, M.; Al Saferan, A.; Al-khaldi, F.; Alhadlaq, F.; Wacaser, B.A., 2014, Sol. Energy, DOI 10.1016/j.solener.2014.08.001',
 'Escribano, M.; Díaz-Caro, C.; Mesias, F.J., 2018, Sci. Total Environ., DOI 10.1016/j.scitotenv.2018.05.297',
 'Gallego Galán, C.; Torralbo, P.; Pérez-Palazón, M.J.; Galán, C.; Polo, M.J., 2019, 21st EGU Gen. Assem. EGU2019',
 'Memiche, M.; Bouzian, C.; Benzahia, A.; Moussi, A., 2020, Glob. Energy Interconnect., DOI 10.1016/j.gloei.2020.03.004',
 'Akbar, S.; Ahmad, T., 2019, Int. J. Eng., DOI 10.34259/ijew.19.605159163',
 'Reinders, A.; Verlinden, P.; van Sark, W.; Freundlich, A., 2017',
 'Bagnold, R.A., 1971, DOI 10.1007/978-94-009-5682-7',
 'Gostein, M.; Littmann, B.; Caron, J.R.; Dunn, L., 2013, 2013 IEEE 39th Photovolt. Spec. Conf, DOI 10.1109/PVSC.2013.6745094',
 'Sarver, T.; Al-Qaraghuli, A.; Kazmerski, L.L., 2013, Renew. Sustain. Energy Rev., DOI 10.1016/j.rser.2012.12.065',
 'Jiang, H.; Lu, L.; Sun, K., 2011

In [5]:
import os, calendar

mode = "a" if os.path.exists(filename) else "w"

if mode == "w":
    with open(filename, mode) as datafile:
        datafile.write("FN Clarivate Analytics Web of Science\nVR 1.0\n")